# Prompt and Response Construction Experiments

## Imports and setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import sys
from pathlib import Path

# Add backend folder to Python path
backend_path = Path.cwd().parent  # Goes from nbs/ to backend/
sys.path.insert(0, str(backend_path))

In [ ]:
from importlib import reload
import app.submodules
import importlib
import app.utils
import app.rag_utils
import app.submodules

In [5]:
from app.submodules import construct_response

## Basic Example

In [6]:
TEXT = "I am located in Vineland, NJ off of Landis Avenue. Do you have information on the soup kitchens in Vineland? I want specific links and addresses"
PREVIOUS_TEXT = []
MODEL = "copilot"
ORG = "cspnj"

In [7]:
generator = construct_response(TEXT, PREVIOUS_TEXT, MODEL, ORG)

In [8]:
full_response = ""
for chunk in generator:
    full_response += chunk.replace("\n\n","").replace("data: ","")

[DEBUG] needs_goals=True, verbosity=deep
[DEBUG] verbosity=deep → using full orchestration with k=50
[DEBUG] copilot pipeline branch (SMART goals + orchestration)


/Users/oliviacheng/projects/research/community_services_llm/backend/app/rag_utils.py:143: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resources_df = pd.read_sql_query("""


All messages [{'role': 'system', 'content': '# Wellbeing SMART Goal Assistant Prompt\n\nYou are a thoughtful and knowledgeable chatbot associated with **cspnj**, designed to help clients with their overall wellbeing.  \nYou guide center members along the **eight dimensions of wellness**: *emotional, physical, occupational, social, spiritual, intellectual, environmental, and financial.*\n\n---\n\n## Task\n\nAt this stage, your role is to **ask purposeful questions** that help clarify the member’s situation and guide them toward meaningful **SMART (Specific, Measurable, Achievable, Realistic, and Timely) goals**.\n\nDo **not** provide resources or direct solutions yet — focus only on gathering the right information.\n\n---\n\n## Guidelines\n\n1. **Ask Insightful, Detailed Questions**  \n   - Include questions about the member’s **location**, **current situation**, and any **context** that would help shape their SMART goals.  \n   - For each question, provide a **brief explanation** of wh

In [9]:
formatted_response = full_response.replace("<br/>","\n")

In [10]:
print(formatted_response)

It seems there was a misunderstanding; I cannot provide specific links or addresses directly. However, I can suggest that you utilize existing resources or set SMART goals around food assistance to help ensure wellbeing for the center member you’re supporting.

In terms of the dimensions of wellness, focusing on **physical wellness** through food security is essential. A SMART goal related to food access could be: 

**Specific:** Obtain food assistance from a local soup kitchen once a week.  
**Measurable:** Track visits and meal options available.  
**Achievable:** Identify two nearby food resources to choose from.  
**Realistic:** Ensure transportation or walking distance to the selected location is manageable.  
**Timely:** Set to start within the next two weeks.

These steps could help improve not only physical wellness but also enhance **social wellness** by creating opportunities to meet others in the community. Would you like to explore any specific goals related to this?[DONE]


In [3]:
import pandas as pd
import os
import psycopg

In [4]:
CONNECTION_STRING = os.getenv('RESOURCE_DB_URL')
with psycopg.connect(CONNECTION_STRING) as conn:
    df = pd.read_sql_query("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """, conn)
    print(df)

/var/folders/lf/lgn5wxd97mzcmfjqdkjmq1dh0000gn/T/ipykernel_61776/602818015.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""


  table_name
0  resources


In [6]:
with psycopg.connect(CONNECTION_STRING) as conn:
    df = pd.read_sql_query("""
        SELECT * from resources
    """, conn)

/var/folders/lf/lgn5wxd97mzcmfjqdkjmq1dh0000gn/T/ipykernel_61776/2679618489.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("""


In [7]:
df.head()

,id,organization,service,description,url,phone,embedding
0,1,cspnj,262-HELP (Crisis Response Service),NaN,NaN,NaN,"[-0.0015057984,-0.04301608,0.015712785,-0.0125..."
1,2,cspnj,(NA) Narcotics Anonymous,Narcotics Anonymous of New Jersey provides sup...,https://www.nanj.org/,8009920401,"[-0.034808174,0.07251281,0.008178203,-0.015189..."
2,3,cspnj,(211) Food Pantries/Soup Kitchens,The NJ 2-1-1 Partnership provides information ...,https://nj211.org/food-pantries-and-soup-kitchens,NaN,"[-0.008280203,0.06520874,-0.0067741363,-0.0098..."
3,4,cspnj,(NA) Recovery,Narcotics Anonymous of New Jersey offers suppo...,https://www.nanj.org/,8009920401,"[-0.027628645,0.08028611,-0.009514642,-0.02050..."
4,5,cspnj,211 Code Blue Alert - NJ Warming Centers,New Jersey Code Blue is a program activated du...,https://nj211.org/njcodeblue,8776521148,"[-0.027639408,-0.0037133705,0.00823962,0.02425..."


In [12]:
for i, s in enumerate(df['service']):
    if "vineland soup kitchen" in s.lower():
        print(df.iloc[i])

id                                                           1232
organization                                                cspnj
service                                     Vineland Soup Kitchen
description     The Vineland Soup Kitchen, operated by Spirit ...
url                          https://www.vinelandsoupkitchen.org/
phone                                                  8566922603
embedding       [-0.017748917,0.031070447,0.012905694,0.036825...
Name: 1234, dtype: object


## Summary of test:

Bot provides correct *resource name* but incorrect *URL* and *phone*.

1. **Vineland Soup Kitchen**
   - **Address:**  200 N 4th St, Vineland, NJ 08360
   - **Phone Number:** (856) 696-1111
   - **Website:** [Vineland Soup Kitchen](http://www.vinelandcity.org/soup_kitchen)

Resource databse contains the same *resource name* but different (and correct) *URL* and *phone*.